In [1]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### get game categories

In [16]:
raw_data = []

url = "https://www.miniclip.com"
response = requests.get(url+"/games")
soup = BeautifulSoup(response.text, 'html.parser')

for x in soup.find('div', class_='category-list').find_all('a', href=True):
    raw_data.append({'category':x['title'].split("Play ")[1], 'url':url+x['href']})
df_cat = pd.DataFrame(raw_data)
df_cat

,category,url
0,2 Player Games,https://www.miniclip.com/games/genre-1301/2-pl...
1,3D Games,https://www.miniclip.com/games/genre-35/3d/en/...
2,Action Games,https://www.miniclip.com/games/genre-13/action...
3,Adventure Games,https://www.miniclip.com/games/genre-515/adven...
4,Alien Games,https://www.miniclip.com/games/genre-1278/alie...
...,...,...
66,Winter Games,https://www.miniclip.com/games/genre-4/winter/...
67,Winter Sports Games,https://www.miniclip.com/games/genre-1292/wint...
68,Word Games,https://www.miniclip.com/games/genre-1109/word...
69,Xplore Games,https://www.miniclip.com/games/genre-1311/xplo...


### get games titles

In [23]:
raw_data = []

for i, row in df_cat.iterrows():    
    response = requests.get(row['url'])
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        games = soup.find('div', id="category-games-list").find('ul', class_='games-list inner').find_all('li')
        print("Category", row['category'])
    except:
        print("Skipping", row['category'])
        continue
    for game in games:
        game_rate = float(game['data-rating'].replace(',', '.'))
        game_rate_count = int(game['data-rating-count'])
        gametitle = game.find('span').text
        gameurl = "https://www.miniclip.com" + game.find('a')['href']
        raw_data.append({'game':gametitle, 
                         'url':gameurl, 
                         'category':row['category'], 
                         'rating': game_rate, 
                         'rating_count': game_rate_count})
        
df_games = pd.DataFrame(raw_data)
df_games.head()

Category 2 Player Games
Category 3D Games
Category Action Games
Category Adventure Games
Category Alien Games
Category American Football Games
Category Android Games
Category Arcade Games
Category Award Games
Category Award Series Games
Category Awesome Games
Category Basketball Games
Category Buzz Games
Category Card Games
Category Christmas Games
Category Crazy Games
Category Credits Games
Category Cricket Games
Category Destruction Games
Category Dress Up Games
Category Driving & Parking Games
Category Extreme Sports Games
Category Fantasy Games
Category Fighting Games
Category Fireboy and Watergirl Games
Category Flying Games
Category Football Games
Category Golf Games
Category Halloween Games
Category Holidays Games
Category Hot Games Games
Category IO Games
Category iPhone Games
Category Latest Games
Category Made in the USA Games
Category Monster Games
Category Motocross Games
Category Motorsports Games
Skipping Multiplayer Games
Category My Friends League Games
Category Platfor

,game,url,category,rating,rating_count
0,Arrow Spam,https://www.miniclip.com/games/arrow-spam/en/#...,2 Player Games,4.1869,1129
1,Battle Golf On,https://www.miniclip.com/games/battle-golf-onl...,2 Player Games,3.9704,3720
2,Alien Transpor,https://www.miniclip.com/games/alien-transport...,2 Player Games,4.4464,1380
3,Soccer Snakes,https://www.miniclip.com/games/soccer-snakes/e...,2 Player Games,3.7380,584
4,Mine Adventure,https://www.miniclip.com/games/miners-adventur...,2 Player Games,4.1738,328


In [24]:
df_games.to_csv("./miniclip_game_info.csv")